Load data in Tensorflow.

In [10]:
import codecs
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [139]:
training_data_folder = '../training_data/web-radio/output/rec'
embDir = '../embeddings'
what = 'artist'

uri_file = '%s/%s.emb.u' % (embDir, what)
vector_file = '%s/%s.emb.v' % (embDir, what)
# header_file = '%s/%s.emb.h' % (embDir, what)
training_file = '%s/%s.dat' % (training_data_folder, what)

vectors = np.array([line.strip().split(' ') for line in codecs.open(vector_file, 'r', 'utf-8')])
# heads = np.array([line.strip() for line in codecs.open(header_file, 'r', 'utf-8')])
uris = np.array([line.strip() for line in codecs.open(uri_file, 'r', 'utf-8')])

train_array = np.array([line.strip().split(' ') for line in codecs.open(training_file, 'r', 'utf-8')])
train_array.shape

(11815, 3)

In [140]:
def get_embs(x):
    v = vectors[np.argwhere(uris == x)]
    if v.size == 0:
        result = -2. * np.ones(vectors[0].size)
    else:
        result = v[0][0]
    return result.astype('float32')

In [141]:
col1 = np.array([get_embs(xi) for xi in train_array[:, 0]])
col2 = np.array([get_embs(xi) for xi in train_array[:, 1]])
col3 = np.array(train_array[:, 2]).astype('float32')
col3 = col3.reshape((col3.size, 1))

training_vector = np.concatenate((col1, col2, col3), axis=1)

In [142]:
train, test = train_test_split(training_vector, train_size=0.3)

train_vector = train[:, :-1]
train_label = train[:, -1]
train_label = train_label.reshape((len(train_label), 1))

test_vector = test[:, :-1]
test_label = test[:, -1]
test_label = test_label.reshape((len(test_label), 1))

/Users/pasquale/anaconda3/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [143]:
print(col1.shape)
print(col2.shape)
print(col3.shape)

print(training_vector.shape)
print(test.shape)

print(train_vector.shape)
print(train_label.shape)

(11815, 14)
(11815, 14)
(11815, 1)
(11815, 29)
(8271, 29)
(3544, 28)
(3544, 1)


In [154]:
# Parameters
learning_rate = 0.4
num_steps = 1000
batch_size = 128
display_step = 100

# Network Parameters
n_hidden_1 = 256  # 1st layer number of neurons
n_hidden_2 = 256  # 2nd layer number of neurons
num_input = train_vector[0].size
num_output = int(num_input / 2)
num_output_wrap = train_label[0].size

# tf Graph input
X = tf.placeholder(tf.float32, [None, num_input], name="Placeholder_X")
Y = tf.placeholder(tf.float32, [None, num_output_wrap], name="Placeholder_Y")

In [155]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([num_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, num_output]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([num_output]))
}

In [156]:
# Create model
def neural_net(x):
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    # Output fully connected layer with a neuron for each class
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    row_sum = tf.reduce_sum(out_layer, axis=1, keepdims=True)
    return tf.divide(out_layer, row_sum)

In [157]:
def weighted_l2(a, b, w):
    # https://stackoverflow.com/a/8861999/1218213
    q = tf.subtract(a, b)
    # return np.sqrt((w * q * q).sum())
    pow_q = tf.cast(tf.pow(q, 2), tf.float32)
    
    return tf.reduce_sum(tf.multiply(w, pow_q), axis=1)

In [158]:
def compute_penalty(expected, taken, total):
    penalty = tf.divide(tf.subtract(expected, taken), total) 
    return tf.cast(penalty, tf.float32)
    
def neural_net_wrap(x, previous_out):
    lt = previous_out.shape.as_list()[0] # vertical size of the tensor
    lh = previous_out[0].shape.as_list()[0] # horizontal size of the tensor
    seed, target = tf.split(x, [lh, lh], axis=1)
    bs = tf.equal(seed, -2.)
    bt = tf.equal(target, -2.)
    
    _ones = tf.ones_like(previous_out, tf.float32)
    max_distance = weighted_l2(_ones, _ones * -1., previous_out)
    
    bad_mask = tf.logical_or(bs, bt)
    good_mask = tf.logical_not(bad_mask)
    
    bs_count = tf.count_nonzero(tf.logical_not(bs), axis=1, keepdims=True)
    good_count = tf.count_nonzero(good_mask, axis=1, keepdims=True)

    _zeros = tf.zeros_like(previous_out, tf.float32)
    _seed = tf.where(good_mask, seed, _zeros)
    _target = tf.where(good_mask, target, _zeros)

    # distance
    d = weighted_l2(_seed, _target, previous_out)
    
    # how much info I am not finding
    penalty = compute_penalty(bs_count, good_count, lh)
    multiplier = tf.subtract(1., penalty)

    # score
    s = tf.divide(tf.subtract(max_distance, d), max_distance)
    return tf.multiply(s, multiplier)

In [159]:
# Construct model
intermediate = neural_net(X)
logits = neural_net_wrap(X, intermediate)

In [160]:
# Define loss and optimizer
# loss_op = similarity_loss(logits=logits, labels=Y)
loss_op = tf.reduce_mean(tf.subtract(logits, Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(logits, Y)
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [161]:
# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [162]:
def next_batch(num, data, labels):

    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0, len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = data[idx]
    labels_shuffle = labels[idx]
    return data_shuffle, labels_shuffle

In [163]:
with tf.Session() as sess:
    # Run the initializer
    sess.run(init)
    print("Start learning")
    for step in range(1, num_steps + 1):
        batch_x, batch_y = next_batch(batch_size, train_vector, train_label)

        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            preds, my_weights, loss, acc = sess.run([logits, intermediate, loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))
#             print("Predictions %s VS %s" % (preds, batch_y))
#             print("My weights %s" % my_weights)

    print("Optimization Finished!")

    print("Testing Accuracy:",
          sess.run(accuracy, feed_dict={X: test_vector, Y: test_label}))

Start learning
Step 1, Minibatch Loss= 0.0018, Training Accuracy= 0.013
Step 100, Minibatch Loss= -0.0228, Training Accuracy= 0.007
Step 200, Minibatch Loss= -0.0338, Training Accuracy= 0.013
Step 300, Minibatch Loss= -0.0131, Training Accuracy= 0.012
Step 400, Minibatch Loss= -0.0292, Training Accuracy= 0.007
Step 500, Minibatch Loss= -0.0028, Training Accuracy= 0.007
Step 600, Minibatch Loss= 0.0123, Training Accuracy= 0.004
Step 700, Minibatch Loss= -0.0085, Training Accuracy= 0.010
Step 800, Minibatch Loss= 0.0126, Training Accuracy= 0.004
Step 900, Minibatch Loss= 0.0195, Training Accuracy= 0.009
Step 1000, Minibatch Loss= -0.0228, Training Accuracy= 0.006
Optimization Finished!
Testing Accuracy: 0.00666715
